In [1]:
pip install langchain requests python-dotenv


In [ ]:
# Install required packages in Colab
!pip install langchain requests -q

import os
import sys
import requests
from getpass import getpass
from langchain.memory import ConversationBufferMemory
from typing import Dict, List

# Prompt user to input the OpenRouter API key securely
API_KEY = getpass("Enter your OpenRouter API_KEY: ")
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Check if API key is provided
if not API_KEY:
    print("Error: No API key provided.")
    sys.exit(1)

# Initialize conversation memory
memory = ConversationBufferMemory(return_messages=True)

def get_model_response(user_input: str, history: List[Dict[str, str]]) -> str:
    """
    Send user input and conversation history to OpenRouter API using google/gemma-3-27b-it:free.
    """
    # Prepare the messages payload
    messages = [{"role": "system", "content": "You are a helpful AI chatbot."}]
    messages.extend(history)
    messages.append({"role": "user", "content": user_input})

    # API request payload with Gemma 3 27B IT (free) model
    payload = {
        "model": "google/gemma-3-27b-it:free",  # Updated to requested free model
        "messages": messages,
        "max_tokens": 500,
        "temperature": 0.7,
    }

    # Headers for authentication
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }

    # Debugging: Print the payload
    print("Debug - Payload:", payload)

    try:
        # Make the API call
        response = requests.post(API_URL, json=payload, headers=headers, timeout=10)
        response.raise_for_status()

        # Parse and return the response
        data = response.json()
        print("Debug - API Response:", data)  # Debugging output
        return data["choices"][0]["message"]["content"].strip()

    except requests.exceptions.HTTPError as e:
        return f"Error: HTTP {e.response.status_code} - {e.response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error: Could not connect to the API - {str(e)}"
    except (KeyError, IndexError):
        return "Error: Invalid response from the API."

def display_history():
    """Display the conversation history."""
    chat_history = memory.chat_memory.messages
    if not chat_history:
        print("No conversation history yet.")
        return

    print("\n--- Conversation History ---")
    for msg in chat_history:
        role = "User" if msg.type == "human" else "AI"
        print(f"{role}: {msg.content}")
    print("--- End of History ---\n")

def main():
    """Run the interactive chatbot."""
    print("Hello! I'm your AI chatbot powered by Gemma 3 27B IT (free). Type 'quit' or 'exit' to stop, or 'history' to see past messages.")

    while True:
        user_input = input("You: ").strip().lower()

        if user_input in ["quit", "exit"]:
            print("Goodbye!")
            break
        elif user_input == "history":
            display_history()
            continue
        elif not user_input:
            print("Please say something!")
            continue

        # Retrieve current conversation history
        history = [
            {"role": "user", "content": msg.content} if msg.type == "human" else {"role": "assistant", "content": msg.content}
            for msg in memory.chat_memory.messages
        ]

        # Get and display response
        response = get_model_response(user_input, history)
        print(f"AI: {response}")

        # Update memory
        memory.save_context({"input": user_input}, {"output": response})

# Run the chatbot
main()

Hello! I'm your AI chatbot powered by Gemma 3 27B IT (free). Type 'quit' or 'exit' to stop, or 'history' to see past messages.
Debug - Payload: {'model': 'google/gemma-3-27b-it:free', 'messages': [{'role': 'system', 'content': 'You are a helpful AI chatbot.'}, {'role': 'user', 'content': 'hi'}], 'max_tokens': 500, 'temperature': 0.7}
Debug - API Response: {'id': 'gen-1743054783-JYsrQfLP42sZMROrDTqE', 'provider': 'Chutes', 'model': 'google/gemma-3-27b-it', 'object': 'chat.completion', 'created': 1743054783, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': "\n\nHi there! 👋 \n\nHow can I help you today? Just let me know what you're thinking, or if you just wanted to say hi, that's great too! I'm here to chat, answer questions, brainstorm ideas, or whatever else you might need. 😊 \n\n\n\n", 'refusal': None}}], 'usage': {'prompt_tokens': 22, 'completion_tokens': 63, 'total_tokens': 85}}
AI: Hi the